In [1]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import scipy.stats as stats

In [2]:
fluence = pd.read_csv('reinserted_thermal_fluence.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'reinserted_thermal_fluence.csv'

In [ ]:
fluence

In [ ]:
def bin_names(arr):
    arr_names = []
    for i in range(len(arr)):
        arr[i] = np.format_float_scientific(arr[i], unique=False, precision=2)
        if i+2 > len(arr):
            break
        else:
            arr[i+1] = np.format_float_scientific(arr[i+1], unique=False, precision=2)
            arr_names.append(str([str(arr[i]) + ':' + str(arr[i+1])]))
    return arr_names

In [ ]:
def fluence_bins(fluence, fluence_col_start, initial_burn_up):
    fluence_only = fluence.iloc[initial_burn_up:,fluence_col_start:]
    timestamp = fluence[['# timestamp']]
    fluence_t = pd.concat([timestamp, fluence_only], axis=1)
    # Read data
    Tbl = fluence_t

    # Extract data
    Xvals = Tbl.iloc[:,1:-1].to_numpy()
    Yvals = Tbl.iloc[:,-1].to_numpy()
    timestamps = Tbl.iloc[:,0].to_numpy()
    nRows, nCols = Xvals.shape
    #Xavg = np.mean(Xvals, axis=1)

    # define bins
    binMax = Tbl.iloc[:,1:].max().max()
    binInt = binMax/200
    hEdges = np.arange(0, binMax+binInt, binInt)
    nBins = len(hEdges)-1

    # fill histogram matrix
    nCountsX = np.zeros((nRows, nBins), dtype=np.uint16)
    for n in range(nRows):
        nCountsX[n,:], _ = np.histogram(Xvals[n,:], hEdges)
        
    counts_df = pd.DataFrame(nCountsX)
    hEdges_name = bin_names(hEdges)
    counts_df.index = timestamps
    counts_df.index.name = 't'
    counts_df.columns = hEdges_name
    return counts_df

In [ ]:
def isotope_bins(data, isotope_col_start, isotope_col_end, initial_burnup, num_bins = 200):
    isotope_only = data.iloc[initial_burnup:,isotope_col_start:isotope_col_end]
    timestamp = data[['# timestamp']].iloc[initial_burnup:]
    Tbl = pd.concat([timestamp, isotope_only], axis=1)

    # Extract data
    Xvals = Tbl.iloc[:,1:-1].to_numpy()
    Yvals = Tbl.iloc[:,-1].to_numpy()
    timestamps = Tbl.iloc[:,0].to_numpy()
    nRows, nCols = Xvals.shape
    #Xavg = np.mean(Xvals, axis=1)

    # define bins
    binMax = Tbl.iloc[:,1:].max().max()
    binInt = binMax/num_bins
    hEdges = np.arange(0, binMax+binInt, binInt)
    nBins = len(hEdges)-1

    # fill histogram matrix
    nCountsX = np.zeros((nRows, nBins), dtype=np.uint16)
    for n in range(nRows):
        nCountsX[n,:], _ = np.histogram(Xvals[n,:], hEdges)
    
    counts_df = pd.DataFrame(nCountsX)
    hEdges_name = bin_names(hEdges)
    counts_df.index = timestamps
    counts_df.index.name = 't'
    counts_df.columns = hEdges_name
    return counts_df

In [ ]:
def concat_isotopes(isotopes,isotope_col_start, isotope_col_end, initial_burnup, num_bins = 200):
    prev_table = False
    df = 0
    for i in range(len(isotopes)):
        if prev_table == False:
            df = isotope_bins(isotopes[i],isotope_col_start[i], isotope_col_end[i], initial_burnup, num_bins)
        else:
            df2 = isotope_bins(isotopes[i],isotope_col_start[i], isotope_col_end[i], initial_burnup, num_bins)
            pd.merge(df, df2, left_index=True, right_index=True)
    return df

In [ ]:
fluence_binned = isotope_bins(fluence, 814, 1627, 50)
cesium_binned = isotope_bins(fluence, 50, 814, 50)

In [ ]:
fluence_binned

In [ ]:
cesium_binned

In [ ]:
fluence_binned.to_csv('fluence.csv')
cesium_binned.to_csv('cesium.csv')